In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml

In [2]:
# Request oldest and newest Season Weeks
response_2010 = requests.get('http://www.nflweather.com/en/week/2010/week-1-2/')
response_2021 = requests.get('http://www.nflweather.com/en/week/2021/week-1/')

In [3]:
weather_2010_soup = BeautifulSoup(response_2010.content, 'html.parser')
weather_2021_soup = BeautifulSoup(response_2021.content, 'html.parser')

In [25]:
# weather_2010_soup.find_all('td', class_='details text-center')#, class_='details text-center footable-last-column')
game_link_2010 = weather_2010_soup.find_all('a', class_='btn btn-success move-rigth')[0]['href']
game_link_2021 = weather_2021_soup.find_all('a', class_='btn btn-success move-rigth')[0]['href']
# test_tbl = weather_2010_soup.find('table')
# pd.read_html(str(test_tbl))[0]

In [16]:
# Request Old Game
response_game_2010 = requests.get(f'http://www.nflweather.com{game_link_2010}')

In [24]:
weather_2010_game_soup = BeautifulSoup(response_game_2010.content, 'html.parser')
weather_2010_game_soup.find_all('div',class_='g-away')[1].get_text().replace('  ',' ')  # away team
weather_2010_game_soup.find_all('div',class_='g-home')[1].get_text().replace('  ',' ')  # home team


'New Orleans Saints'

In [36]:
(weather_2010_game_soup.find_all('div',class_='span3')[1].p) == None

True

In [28]:
# Request New Game
game_link_all = f'http://www.nflweather.com{game_link_2021}'
print(f'Requesting {game_link_all}')
response_game_2021 = requests.get(game_link_all)

Requesting http://www.nflweather.com/en/game/2021/week-1/cowboys-at-buccaneers


In [30]:
weather_2021_game_soup = BeautifulSoup(response_game_2021.content, 'html.parser')
weather_2021_game_soup.find_all('div',class_='g-away')[1].get_text().replace('  ',' ')  # away team
weather_2021_game_soup.find_all('div',class_='g-home')[1].get_text().replace('  ',' ')  # home team

'Tampa Bay Buccaneers'

In [110]:
from src import helpers
import importlib
importlib.reload(helpers)
# Check if 2nd Quarter Exists - break loop on four quarters if the next is None
quarter_1 = weather_2021_game_soup.find_all('div',class_='span3')[0]
q1_weather_elements = quarter_1.find_all('p')
print(q1_weather_elements[1].get_text())  # Quarter
print(q1_weather_elements[3].get_text().strip())  # Weather Conditions
# Temperature
tf, el = helpers.check_for_element('Temperature', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('f.',''))  # Temperature in DegF
# Feels Like
tf, el = helpers.check_for_element('Feels Like', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('f.',''))  # Feels Like in DegF
# Wind
tf, el = helpers.check_for_element('Wind', q1_weather_elements)
if tf:
    wind_els = el.b.get_text().split('mi ')
    print(wind_els[0])  # Wind Speed in mph
    print(wind_els[1])  # Wind Direction
# Humidity
tf, el = helpers.check_for_element('Humidity', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('%',''))  # Humidity PCT
# Visibility
tf, el = helpers.check_for_element('Visibility', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('mi',''))  # Visibility in Miles
# Barometer
tf, el = helpers.check_for_element('Barometer', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('"',''))  # Barometer in Miles
# Dew Point
tf, el = helpers.check_for_element('Dew Point', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('f.',''))  # Dew Point in DegF
# Cloud Cover
tf, el = helpers.check_for_element('Cloud Cover', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('%',''))  # Cloud Cover PCT
# Precipitation Prob
tf, el = helpers.check_for_element('Precipitation', q1_weather_elements)
if tf:
    print(el.b.get_text().replace('%',''))  # Precipitation Prob PCT

## Start of Stadium Details
stadium_details = weather_2021_game_soup.find_all('div',class_='span4')[1]
stadium_details_elements = stadium_details.find_all('p')
# Stadium Name
print(stadium_details_elements[2].get_text())
# Stadium Surface
tf, el = helpers.check_for_element('Surface', stadium_details_elements)
if tf:
    print(el.b.get_text())
# Stadium Location
tf, el = helpers.check_for_element('Location', stadium_details_elements)
if tf:
    print(el.b.get_text())
# Stadium Zip
tf, el = helpers.check_for_element('Zip', stadium_details_elements)
if tf:
    print(el.b.get_text())
# Stadium Type
tf, el = helpers.check_for_element('Type', stadium_details_elements)
if tf:
    print(el.b.get_text())



Kickoff
Humid and Partly Cloudy
80
87
6
West - SouthWest
85
10
1012
75
45
0
Raymond James Stadium
Grass
Tampa, Florida
33607
Open


In [130]:
from src import helpers
from src import scrapers
import importlib
importlib.reload(helpers)
importlib.reload(scrapers)
nflw_scraper = scrapers.NflwScraper()
# df_2021 = nflw_scraper.get_games(seasons=2010, weeks=1, game_limit=1)
week_nums, week_labels, week_hrefs= helpers.weeks_lkp(2010)
